In [6]:
import torch
import abmap

In [7]:
# Load AbMAP 
# Using Bepler-Berger as foundational model
abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_beplerberger_H.pt', plm_name='beplerberger')
abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_beplerberger_L.pt', plm_name='beplerberger')

# Using ESM1b
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm1b_H.pt', plm_name='esm1b')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm1b_L.pt', plm_name='esm1b')

# Using ESM2
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm2_H.pt', plm_name='esm2')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm2_L.pt', plm_name='esm2')

# Using ProtBert
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_protbert_H.pt', plm_name='protbert')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_protbert_L.pt', plm_name='protbert')

beplerberger loaded to cpu
Loaded the Pre-trained Model!
beplerberger loaded to cpu
Loaded the Pre-trained Model!


In [9]:
abmap.abmap_embed?

Signature:
abmap.abmap_embed(
    device,
    pretrained_path,
    chain_type,
    input_file,
    output_dir,
    variable_length,
    plm_name,
    task,
)
Docstring:
Given fasta sequences and a pre-trained AbMAP model,
generate their AbMAP embeddings (fixed or variable).

***
For Pre-Trained AbMAP models, the augmented embeddings have the following parameters:
    Foundational PLM: Bepler & Berger
    CDR-masks (4 extra dims)
    NO separators
    # mutations = 100
File:      ~/opt/anaconda3/envs/abmap_main/lib/python3.9/site-packages/abmap/commands/embed.py
Type:      function

In [8]:
# Get embedding for one sequence
demo_seq = 'EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS'

abmap_H.abmap_
# Contrastive augmentation (PLM, mutagenesis, CDR focus)
x = abmap.ProteinEmbedding(demo_seq, chain_type='H', embed_device=torch.device('cpu'))
x.create_cdr_specific_embedding(embed_type='beplerberger', k=50)

# Pass the augmented embedding to AbMAP to get final embedding
with torch.no_grad():
    embed_var = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='variable') # residue-level embeddings
    embed_fl = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='fixed') # fixed-length

../data/anarci_files/temp0
Sequence: EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS
scheme: chothia


sh: ANARCI: command not found


IndexError: list index out of range

In [5]:
embed_var.shape, embed_fl.shape

NameError: name 'embed_var' is not defined

In [ ]:
# Get embeddings for a fasta file of sequences
# COMING VERY SOON

In [6]:
# Use ESM2 as foundational model
# COMING SOON

In [ ]:
# Fine-tuning AbMAP on your own functional data
# COMING SOON